

### Infering Force Field Parameters from a Trajectory
The availability of automatic differention (AD) within a molecular dynamics package is benefitial beyond ML applications.
Being able to compute gradients for all numerical operations opens up new avenues for sensitivity analysis, force field optimization, steered MD simulations, as well as simulations under highly complex constraints and restraints. To demonstrate these capabilities, the present example infers force field parameters from a short MD trajectory.

First, a small water box containing 97 water molecules and one Na$^+$/Cl$^-$ ion pair was simulated using the TIP3P water model with flexible bonds and angles. After energy minimization and NVT equilibration at 300 K, the simulation was run for 10 ps in the microcanonical ensemble. The simulation used a 1 fs time step, a 9 A cutoff with 7.5 A switch distance, and reaction field electrostatics. Coordinates and velocities were saved every 10 steps.

Next, all partial atomic charges, $q,$ in the system were annihilated (in practice, they were scaled by 0.01 to ensure non-vanishing gradients of the electrostatic potential). In order to reobtain $q$ from the MD trajectory, the integrator was initialized with snapshots $r(t_i)$, $v(t_i)$ from the trajectory. Then, 10 steps of simulation were run with the modified charges and the final positions from this micro-simulation were compared with the respective subsequent trajectory snapshot $r(t_{i+1})$. In other words, this *micro-simulation* served as a parameterized propagator
$Q: (r(t), v(t), q) \mapsto r(t + \delta_t)$ with $\delta_t = 10$ fs. Due to the AD capabilities, this propagator is end-to-end differentiable. 

To reobtain the charges, we minimized the loss function
$$
L(r(t_i), v(t_i); q) = \| Q(r(t_i), v(t_i), q) - r(t_{i+1}) \|_2^2,
$$

i.e. the mean-squared distance between the ground-truth trajectory and the propagated coordinates (taking into account periodic boundary conditions). This loss function is differentiable with respect to the charges $q$ and gradients are obtained via backpropagation. Training was performed using Adam with a learning rate of $10^{-3}$ over one snapshot at a time. To enforce net neutrality, the positive charges ($q_H$ and $q_{\mathrm{Na}^+}$) were obtained from the oxygen and chlorine charges. Figure ... shows the train loss and the partial atomic charges during training. After just two epochs (2000 Iterations), the original charges were recovered up to 2% accuracy.


<img width="400" alt="portfolio_view" src="loss_water.png">
<img width="400" alt="portfolio_view" src="charges_water.png">
